# -----------------PROYECTO FINAL-----------------

## 1. IMPORTAR LIBRERIAS

In [1]:
from keras.src.saving.saving_api import load_model  # ⚠️ Import no estándar de Keras, otra "from tensorflow.keras.models import load_model"
import cv2  # OpenCV para procesamiento de imágenes/video
import mediapipe as mp  # Framework para detección de posturas corporales
import numpy as np  # Manejo de arrays numéricos
import os  # Interacción con sistema operativo
import tensorflow as tf  # Framework de ML
from collections import deque  # Estructura de datos tipo cola
from sklearn.model_selection import train_test_split  # División de datasets

import socket  # Comunicación en red
import threading  # Ejecución paralela

import psutil  # Monitoreo de recursos del sistema
import time  # Medición de tiempos/FPS


## 2. INICIALIZAR MEDIAPIPE

In [2]:
class CircularBuffer:
    def __init__(self, size):
        # Inicializa buffer con array numpy pre-dimensionado
        #self.buffer = np.zeros((size), dtype=np.float32)  
        self.buffer = np.zeros((size, total_landmarks), dtype=np.float32) #para que funcione ponerlo en los parametros y modificar evaluation, ahi sigue la explicacion 
        self.size = size  # Capacidad máxima del buffer
        self.idx = 0  # Puntero de escritura actual
        self.full = False  # Flag de buffer lleno 

    def add(self, data):
        # Almacena datos en posición actual y actualiza índice
        self.buffer[self.idx] = data  # Escritura en posición del puntero
        self.idx = (self.idx + 1) % self.size  # Incremento circular
        self.full = self.full or self.idx == 0  # Actualiza estado de llenado

    def get_sequence(self):
        # Devuelve secuencia ordenada temporalmente
        if self.full:
            # Buffer lleno: concatena final + inicio
            return np.concatenate([self.buffer[self.idx:], self.buffer[:self.idx]])
        return self.buffer[:self.idx]  # 🚨 Secuencia invertida si no está lleno

#class del me_12_4 va con server-UDPv3,4,5,8
class UDPCamera:
    def __init__(self):
        self.host = '0.0.0.0'
        self.port = 5000
        self.buffer_size = 131072 #128kb
        self.mtu = 1400  # Tamaño máximo de fragmento
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        self.sock.settimeout(2)
        self.frame = None
        self.fragments = []  # Almacena fragmentos del frame
        self.full = False
        self.running = False
        self.thread = None
        self.lock = threading.Lock()  # Para acceso thread-safe
        self.start()

    def start(self):
        if not self.running:
            self.running = True
            self.sock.bind((self.host, self.port))
            
            self.thread = threading.Thread(target=self._receive_frames, daemon=True)
            self.thread.start()

    def _receive_frames(self):
        while self.running:
            try:
                # Recibir fragmento
                fragment, _ = self.sock.recvfrom(self.buffer_size)
                
                with self.lock:
                    self.fragments.append(fragment)

                    # Detectar último fragmento (tamaño < MTU)
                    if len(fragment) < self.mtu:
                        # Reconstruir frame completo
                        frame_bytes = b''.join(self.fragments)
                        self.fragments = []  # Resetear fragmentos

                        # Decodificar y almacenar frame
                        frame_array = np.frombuffer(frame_bytes, dtype=np.uint8)
                        self.frame = cv2.imdecode(frame_array, cv2.IMREAD_COLOR)

            except socket.timeout:
                #print("[WARN] Timeout esperando fragmentos UDP") 
                continue
            except Exception as e: 
                print(f"[ERROR] Recepción UDP: {str(e)}")
                break

    def read(self):
        with self.lock:
            if self.frame is not None:
                return True, self.frame.copy()  # Copia para thread-safe
            return False, None

    def isOpened(self):
        return self.running

    def release(self):
        self.running = False
        with self.lock:
            self.fragments = []
            self.frame = None
        if self.thread and self.thread.is_alive():
            self.thread.join(timeout=1)
        self.sock.close()
        print("Cámara UDP liberada")

    def __del__(self):
        self.release()

In [3]:
#class del me_12_4_retoque va con server-UDPv6 y v7
class UDPCamera: #Se define la clase UDPCamera, que implementa un cliente de cámara utilizando el protocolo UDP.    
    """
    Enhanced UDP camera client that receives video frames over UDP with improved
    reliability, frame management, and performance.
    """
    #def __init__(self, host='0.0.0.0', port=5000, buffer_size=65536, timeout=0.1): 
    def __init__(self, host='0.0.0.0', port=5000, buffer_size=131072, timeout=0.1): 
        #Se define el método constructor __init__
        #el cual permite inicializar una instancia de UDPCamera con parámetros configurables:
        """
        Initialize the UDP camera with configurable parameters.
        
        Args:
            host: IP address to bind to
            port: UDP port to listen on
            buffer_size: Maximum UDP packet size to receive
            timeout: Socket timeout in seconds
        """
        self.host = host #Se asigna el valor del parámetro host al atributo de la instancia self.host
        self.port = port #Se asigna el valor del parámetro port al atributo self.port.
        self.buffer_size = buffer_size #Se asigna el valor del parámetro buffer_size al atributo self.buffer_size.
        
        # Socket configuration:
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM) 
        #Se crea un socket UDP utilizando la familia de 
        #direcciones IPv4 (AF_INET)
        #el tipo de socket datagrama (SOCK_DGRAM). Este socket se asigna al atributo self.sock.

        self.sock.setsockopt(socket.SOL_SOCKET, socket.SO_RCVBUF, 4 * buffer_size)  
        #Se configura la opción del socket para ampliar el tamaño del buffer de recepción. 
        # Se utiliza setsockopt con el nivel SOL_SOCKET y la opción SO_RCVBUF para establecer 
        # un buffer 4 veces mayor que el tamaño indicado en buffer_size, 
        # lo cual ayuda a recibir paquetes grandes o múltiples paquetes sin perder datos.

        self.sock.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)  # Permitir reusar el puerto (NUEVO EXPERIMENTAL)


        self.sock.settimeout(timeout)
        #Se establece un timeout en el socket con el valor del parámetro timeout,
        #lo que permite que las operaciones de recepción sean más ágiles y evita bloqueos prolongados.

        
        # Frame management
        self.current_frame = None #esta variable almacenará el frame actualmente reconstruido.
        self.last_frame = None  # Esta variable guarda el último frame válido recibido, sirviendo como respaldo en caso de errores o ausencia de nuevos frames.
        self.frame_fragments = {}  #diccionario vacío. Se utilizará para almacenar fragmentos de datos de cada frame, indexados por el identificador del frame.
        self.frame_timestamps = {}  #diccionario vacío, para registrar el momento en que se empezó a recibir cada frame. Esto ayuda a gestionar tiempos de espera y descartar frames incompletos.
        self.fragment_count = 0 #contará el número de fragmentos recibidos para el frame actual.
        
        # Thread control
        self.running = False #Esta bandera indicará si el hilo de recepción está activo.
        self.thread = None #más adelante contendrá el objeto del hilo encargado de recibir los frames.
        self.lock = threading.Lock() #Se crea un objeto Lock a través de threading.Lock() y se asigna a self.lock. Este lock se usará para sincronizar el acceso a variables compartidas entre hilos.
        self.fps = 0 #la variable que almacenará la tasa de frames por segundo (FPS) calculada.
        self.last_frame_time = time.time() #Se guarda el tiempo actual en self.last_frame_time mediante time.time(), para posteriormente calcular la diferencia de tiempo entre frames.
        self.frame_count = 0 #llevará el conteo de frames recibidos.
        self.frame_times = deque(maxlen=30)  # For FPS calculation
        #Se crea una estructura deque con un máximo de 30 elementos asignada a self.frame_times. 
        # Este deque se utilizará para almacenar los tiempos de recepción de los últimos frames 
        # y calcular el FPS de forma promedio.

        
        # Start receiving thread
        self.start() 
        #Se llama al método self.start(), el cual se encargará de iniciar el proceso en segundo plano 
        # para recibir y reconstruir los frames entrantes por UDP.
    
    def start(self):  # Define el método start de la clase UDPCamera, encargado de iniciar el hilo receptor UDP
        
        """Start the UDP receiver thread."""  # Docstring que describe la función del método
        if not self.running:  # Verifica si el receptor UDP aún no está en ejecución (self.running es False)
            try:  # Inicia un bloque try para capturar posibles errores al iniciar el receptor
                self.sock.bind((self.host, self.port))  # Asocia (bind) el socket a la dirección IP (self.host) y puerto (self.port) configurados
                self.running = True  # Marca el receptor como activo estableciendo self.running a True
                self.thread = threading.Thread(target=self._receive_frames, daemon=True)  # Crea un nuevo hilo que ejecutará el método _receive_frames; el hilo es daemon para que se cierre al finalizar el programa principal
                self.thread.start()  # Inicia la ejecución del hilo creado para la recepción de frames
                print(f"UDP Camera started on {self.host}:{self.port}")  # Imprime un mensaje en consola indicando que la cámara UDP ha iniciado correctamente en la dirección y puerto especificados
            except Exception as e:  # Captura cualquier excepción que pueda ocurrir durante la inicialización del receptor
                print(f"Failed to start UDP camera: {e}")  # Imprime un mensaje de error detallando la excepción encontrada

    
    def _receive_frames(self):  # Método privado que se ejecuta en un hilo para recibir y reconstruir frames de video desde paquetes UDP
        """Background thread that receives and reconstructs video frames from UDP packets."""  # Docstring que describe la función del método
        frame_timeout = 0.5  # Define el tiempo máximo (0.5 segundos) para recibir todos los fragmentos de un frame
        
        while self.running:  # Bucle que se ejecuta continuamente mientras la cámara esté en funcionamiento
            try:  # Intenta ejecutar el siguiente bloque, capturando errores si ocurren
                # Receive a packet
                packet_data, _ = self.sock.recvfrom(self.buffer_size)  # Recibe un paquete UDP con un tamaño máximo definido por buffer_size
                
                # First 8 bytes: frame_id (4 bytes) + fragment_id (2 bytes) + total_fragments (2 bytes)
                if len(packet_data) < 8:  # Verifica que el paquete tenga al menos 8 bytes (el tamaño mínimo del encabezado)
                    continue  # Si el paquete es demasiado corto, se ignora y se continúa con la siguiente iteración
                
                header = packet_data[:8]  # Extrae los primeros 8 bytes del paquete, que contienen el encabezado con identificadores
                frame_id = int.from_bytes(header[0:4], byteorder='big')  # Convierte los primeros 4 bytes en un entero que representa el ID del frame, usando orden big-endian
                fragment_id = int.from_bytes(header[4:6], byteorder='big')  # Convierte los siguientes 2 bytes en un entero que representa el ID del fragmento
                total_fragments = int.from_bytes(header[6:8], byteorder='big')  # Convierte los últimos 2 bytes en un entero que indica el número total de fragmentos que componen el frame
                
                # Image data starts after header
                fragment_data = packet_data[8:]  # Extrae los datos de imagen que siguen al encabezado de 8 bytes
                
                with self.lock:  # Adquiere un bloqueo para garantizar el acceso seguro a las variables compartidas entre hilos
                    # Initialize tracking for new frame
                    if frame_id not in self.frame_fragments:  # Si el frame_id aún no existe en el diccionario de fragmentos
                        self.frame_fragments[frame_id] = {}  # Inicializa un diccionario para almacenar los fragmentos de este frame
                        self.frame_timestamps[frame_id] = time.time()  # Registra el tiempo actual como el inicio de la recepción de este frame
                    
                    # Store this fragment
                    self.frame_fragments[frame_id][fragment_id] = fragment_data  # Almacena el fragmento recibido, usando fragment_id como clave
                    
                    # Check if we have all fragments for this frame
                    if len(self.frame_fragments[frame_id]) == total_fragments:  # Si se han recibido tantos fragmentos como se esperaba para el frame
                        # Reconstruct the complete frame
                        frame_data = b''  # Inicializa una variable de bytes para concatenar los fragmentos y reconstruir el frame
                        for i in range(total_fragments):  # Itera sobre el rango de fragmentos esperados (de 0 a total_fragments-1)
                            if i in self.frame_fragments[frame_id]:  # Verifica que el fragmento con índice i esté presente
                                frame_data += self.frame_fragments[frame_id][i]  # Añade el fragmento al frame_data en orden
                            else:
                                # Missing fragment, discard frame
                                break  # Si falta algún fragmento, sale del bucle y se descarta la reconstrucción del frame
                        else:  # Este bloque se ejecuta solo si no se interrumpe el bucle (es decir, si se recibieron todos los fragmentos)
                            # All fragments received, decode frame
                            try:  # Intenta decodificar el frame reconstruido
                                frame_array = np.frombuffer(frame_data, dtype=np.uint8)  # Convierte los datos binarios en un array de NumPy de tipo uint8
                                decoded_frame = cv2.imdecode(frame_array, cv2.IMREAD_COLOR)  # Decodifica el array en una imagen a color usando OpenCV
                                
                                if decoded_frame is not None:  # Si la imagen se decodificó correctamente
                                    # Successfully decoded a valid frame
                                    self.last_frame = self.current_frame  # Actualiza last_frame con el frame actual antes de reemplazarlo
                                    self.current_frame = decoded_frame  # Establece el frame decodificado como el frame actual
                                    
                                    # Calculate FPS
                                    now = time.time()  # Obtiene el tiempo actual para el cálculo del FPS
                                    self.frame_times.append(now)  # Agrega el tiempo actual a la lista de tiempos de frames
                                    if len(self.frame_times) >= 2:  # Si hay al menos dos tiempos registrados
                                        self.fps = len(self.frame_times) / (self.frame_times[-1] - self.frame_times[0])  # Calcula los FPS dividiendo el número de frames por la diferencia de tiempo entre el primer y el último frame registrado
                            except Exception as e:  # Captura excepciones que puedan ocurrir durante la decodificación del frame
                                print(f"Error decoding frame: {e}")  # Imprime un mensaje de error con la descripción de la excepción
                        
                        # Clean up this frame's fragments regardless of success
                        del self.frame_fragments[frame_id]  # Elimina los fragmentos almacenados para el frame procesado, liberando memoria
                        del self.frame_timestamps[frame_id]  # Elimina el timestamp asociado al frame procesado
                
                # Clean up old incomplete frames
                current_time = time.time()  # Obtiene el tiempo actual para comparar con los timestamps de frames incompletos
                with self.lock:  # Adquiere nuevamente el bloqueo para manipular datos compartidos de manera segura
                    expired_frames = [fid for fid, ts in self.frame_timestamps.items() 
                                     if current_time - ts > frame_timeout]  # Crea una lista de IDs de frames cuyo tiempo transcurrido supera el frame_timeout, indicando que están incompletos
                    for frame_id in expired_frames:  # Itera sobre cada frame que ha excedido el tiempo límite
                        del self.frame_fragments[frame_id]  # Elimina los fragmentos del frame expirado
                        del self.frame_timestamps[frame_id]  # Elimina el timestamp del frame expirado
            except Exception as e:  # Captura excepciones generales que puedan ocurrir en el bloque principal de recepción
                print(f"Error in UDP receiver: {e}")  # Imprime un mensaje de error indicando la falla en la recepción

            
            except socket.timeout:
                # This is normal, just continue
                pass
            except Exception as e:
                print(f"Error in UDP receiver: {e}")
                if not self.running:
                    break
    
    def read(self):  # Define el método 'read' que se utiliza para obtener el último frame completo recibido
        """
        Read the latest complete frame.
        
        Returns:
            tuple: (success, frame) where success is True if a frame is available
        """  # Docstring que explica que el método retorna una tupla (éxito, frame) donde éxito es True si hay un frame disponible
        with self.lock:  # Adquiere el bloqueo (lock) para asegurar que el acceso a los frames sea thread-safe
            if self.current_frame is not None:  # Verifica si hay un frame actual disponible
                return True, self.current_frame.copy()  # Devuelve True y una copia del frame actual para evitar modificaciones accidentales
            elif self.last_frame is not None:  # Si no hay un frame actual, verifica si existe un último frame válido (fallback)
                # Return last good frame as fallback
                return True, self.last_frame.copy()  # Devuelve True y una copia del último frame válido, como respaldo
            return False, None  # Si ninguno de los frames está disponible, retorna False y None

    
    def get_fps(self):  # Método para obtener los FPS (frames por segundo) estimados actualmente
            """Get the current estimated FPS."""  # Docstring que explica que retorna los FPS estimados
            with self.lock:  # Adquiere el lock para asegurar acceso thread-safe a la variable compartida
                return self.fps  # Retorna el valor actual de FPS

    def isOpened(self):  # Método que verifica si la cámara está en funcionamiento
            """Check if the camera is running."""  # Docstring que indica que verifica el estado de la cámara
            return self.running and self.thread and self.thread.is_alive()  # Retorna True si 'running' es True, existe un hilo y dicho hilo está activo

    def release(self):  # Método para detener el hilo receptor y liberar los recursos asociados
            """Stop the receiver thread and release resources."""  # Docstring que explica el propósito del método
            self.running = False  # Detiene el proceso de recepción, marcando 'running' como False
            if self.thread and self.thread.is_alive():  # Verifica si existe un hilo y si este sigue activo
                self.thread.join(timeout=1)  # Espera hasta 1 segundo a que el hilo finalice su ejecución
            
            with self.lock:  # Adquiere el lock para modificar de forma segura las variables compartidas
                self.frame_fragments.clear()  # Limpia el diccionario de fragmentos de frames
                self.frame_timestamps.clear()  # Limpia el diccionario de timestamps asociados a cada frame
                self.current_frame = None  # Resetea el frame actual a None
                self.last_frame = None  # Resetea el último frame válido a None
            
            try:  # Intenta cerrar el socket
                self.sock.shutdown(socket.SHUT_RDWR)  # Cierra la conexión del socket (NUEVO EXPERIMENTAL)
                self.sock.close()  # Cierra el socket UDP para liberar el recurs    
            except:  # Si ocurre algún error al cerrar el socket
                pass  # Ignora el error y continúa sin interrumpir el flujo
            
            print("UDP Camera released")  # Imprime un mensaje indicando que la cámara UDP ha sido liberada
 
    def __del__(self):  # Método destructor que se llamaría al eliminar la instancia (comentado para no ejecutarse automáticamente)
        self.release()  # Llama al método release para asegurarse de liberar los recursos al eliminar el objeto


In [4]:
# Configuración inicial global
mp_hands = mp.solutions.hands  # Se asigna el módulo de detección de manos de MediaPipe a la variable 'mp_hands'

# Optimizar MediaPipe
hands = mp_hands.Hands(  # Se crea una instancia del detector de manos con parámetros optimizados para video en tiempo real
    static_image_mode=False,  # Se desactiva el modo de imagen estática para permitir el procesamiento continuo de video
    max_num_hands=2,  # Se limita la detección a un máximo de 2 manos por frame
    min_detection_confidence=0.45,  # Se establece la confianza mínima para la detección de manos (valor reducido para mayor velocidad)
    min_tracking_confidence=0.45,  # Se establece la confianza mínima para el seguimiento de manos entre frames
    model_complexity=0  # Se utiliza un modelo de menor complejidad para acelerar el procesamiento
)

mp_draw = mp.solutions.drawing_utils  # Se asigna la utilidad de dibujo de MediaPipe a 'mp_draw' para dibujar los landmarks en los frames

"""
MODEL_PATH = "model_quantized_90_4_modelo_del_v3.tflite"
NORMALIZATION_PARAMS_PATH = 'normalization_params_90_4_modelo_del_v3.npz'
dataset_dir = "dataset_11_90"  # Se define el directorio donde se guardarán las secuencias de datos (dataset)
model_path = "gesture_model_me_12_90_4_modelo_del_v3.h5"  # Se especifica la ruta y nombre del archivo del modelo de reconocimiento de gestos
"""
MODEL_PATH = "model_quantized_90_4.tflite"
NORMALIZATION_PARAMS_PATH = 'normalization_params_90_4.npz'
dataset_dir = "dataset_11_90"  # Se define el directorio donde se guardarán las secuencias de datos (dataset)
model_path = "gesture_model_me_12_90_4.h5"  # Se especifica la ruta y nombre del archivo del modelo de reconocimiento de gestos



sequence_length = 90  # Se define la longitud de cada secuencia (número de frames por secuencia) para el análisis de gestos
total_landmarks = 126  # Se establece el número total de landmarks (puntos de referencia) que se esperan por secuencia (por ejemplo, 21 puntos por mano * 3 coordenadas x2 manos = 126)
gestures = []  # Se inicializa una lista vacía que posteriormente contendrá los nombres de los gestos disponibles
X_mean = None  # Se inicializa la variable para la media de los datos, la cual se usará para normalizar las secuencias
X_std = None  # Se inicializa la variable para la desviación estándar de los datos, para normalización de las secuencias


## 3. FUNCIONES PRINCIPALES

In [5]:
# Funciones principales
def init_system():  # Función que inicializa el sistema preparando el entorno de trabajo
    global gestures  # Declara que se usará la variable global 'gestures' para actualizar la lista de gestos disponibles
    os.makedirs(dataset_dir, exist_ok=True)  # Crea el directorio definido en 'dataset_dir' si no existe (exist_ok=True evita errores si ya existe)
    gestures = get_existing_gestures()  # Asigna a 'gestures' la lista de gestos existentes obtenida de la función get_existing_gestures()

def get_existing_gestures():  # Función que retorna una lista ordenada de gestos existentes en el directorio del dataset
    return sorted([  # Devuelve la lista ordenada alfabéticamente
        d for d in os.listdir(dataset_dir)  # Itera sobre cada elemento 'd' en el listado de archivos y carpetas del directorio 'dataset_dir'
        if os.path.isdir(os.path.join(dataset_dir, d))  # Incluye 'd' en la lista solo si es un directorio (lo que indica un gesto)
    ])


## 4. DETECCION DE MANO

In [6]:
def detect_hands():  # Define la función detect_hands para iniciar la detección de manos en tiempo real
    print("\nIniciando detección de manos. Presiona 'ESC' para salir.")  # Imprime un mensaje informativo al usuario sobre el inicio del proceso y cómo salir
    cap = UDPCamera()  # Crea una instancia de UDPCamera para capturar frames de video vía UDP

    while True:  # Inicia un bucle infinito para procesar continuamente los frames capturados
        ret, frame = cap.read()  # Llama al método read() para obtener el frame actual; ret indica éxito y frame contiene la imagen
        if not ret:  # Si no se pudo leer un frame correctamente
            continue  # Omite la iteración actual y vuelve a intentar leer un nuevo frame

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convierte el frame de BGR (formato de OpenCV) a RGB (requerido por MediaPipe)
        results = hands.process(rgb_frame)  # Procesa el frame convertido con MediaPipe para detectar landmarks de manos

        if results.multi_hand_landmarks:  # Si se han detectado landmarks en el frame
            for hand_landmarks in results.multi_hand_landmarks:  # Itera por cada conjunto de landmarks detectados (una por mano)
                mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)  # Dibuja los landmarks y las conexiones en el frame usando las utilidades de MediaPipe

        cv2.imshow("Detección de Manos", frame)  # Muestra el frame procesado en una ventana titulada "Detección de Manos"
        if cv2.waitKey(1) & 0xFF == 27:  # Espera 1 milisegundo por una tecla; si se detecta la tecla ESC (código 27), se sale del bucle
            break  # Rompe el bucle infinito para detener la detección de manos

    cap.release()  # Libera los recursos de la cámara UDP al finalizar la captura
    cv2.destroyAllWindows()  # Cierra todas las ventanas abiertas por OpenCV


## 5. RECOLLECION DE DATOS 

In [7]:
def collect_data():
    global gestures  # Accede a la variable global 'gestures' para actualizarla posteriormente
    gesture = input("\nIngrese la palabra o letra para la cual desea recolectar datos: ").upper()  # Solicita al usuario el nombre del gesto y lo convierte a mayúsculas
    num_sequences = int(input("Ingrese el número de secuencias a capturar (recomendado: 50): "))  # Define cuántas secuencias de video se capturarán
    
    save_dir = os.path.join(dataset_dir, gesture)  # Crea la ruta del directorio donde se guardarán los datos
    os.makedirs(save_dir, exist_ok=True)  # Crea el directorio si no existe, evitando errores si ya está presente

    print(f"\nRecolectando datos para el gesto '{gesture}'. Presiona 'ESC' para cancelar.")  # Mensaje informativo
    print("Mantenga la seña frente a la cámara...")  # Instrucción para el usuario
    
    cap = UDPCamera()  # Inicializa la cámara UDP personalizada
    sequence = []  # Lista para almacenar temporalmente los datos de landmarks de cada secuencia
    counter = 0  # Contador de secuencias capturadas

    # Configurar ventana de landmarks
    landmark_window_name = "Landmarks en Tiempo Real"  # Nombre de la ventana auxiliar
    cv2.namedWindow(landmark_window_name, cv2.WINDOW_NORMAL)  # Crea una ventana redimensionable
    cv2.resizeWindow(landmark_window_name, 640, 480)  # Establece el tamaño de la ventana


    while True:  # Bucle principal de captura
        ret, frame = cap.read()  # Lee un frame de la cámara
        if not ret:  # Si falla la lectura, sale del bucle
            break

        # Crear canvas para landmarks
        landmark_canvas = np.zeros((480, 640, 3), dtype=np.uint8)  # Crea una imagen negra de 640x480 píxeles

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convierte el frame de BGR a RGB para MediaPipe
        results = hands.process(rgb_frame)  # Procesa el frame para detectar landmarks de manos

        if results.multi_hand_landmarks:  # Si se detectan manos:
            all_landmarks = []  # Lista para almacenar coordenadas de landmarks

            # Dibujar landmarks en el canvas
            for hand_landmarks in results.multi_hand_landmarks:  # Itera sobre cada mano detectada
                # Dibuja landmarks y conexiones en el canvas negro
                mp_draw.draw_landmarks(
                    landmark_canvas,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,  # Especifica las conexiones anatómicas
                    mp_draw.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2),  # Configuración visual de puntos
                    mp_draw.DrawingSpec(color=(0, 0, 255), thickness=2)  # Configuración visual de líneas
                )
            
            # Extraer coordenadas para el dataset
            for hand in results.multi_hand_landmarks[:2]:  # Considera máximo 2 manos
                for lm in hand.landmark:  # Itera sobre cada landmark de la mano
                    all_landmarks.extend([lm.x, lm.y, lm.z])  # Almacena coordenadas normalizadas (x,y,z)
            
            # Rellenar si solo hay una mano
            if len(results.multi_hand_landmarks) < 2:  # Si hay menos de 2 manos detectadas
                all_landmarks += [0.0] * 63  # Añade 63 ceros (21 landmarks * 3 coordenadas por mano faltante)
            
            sequence.append(all_landmarks)

            # Dibujar en el frame original
            for hand_landmarks in results.multi_hand_landmarks:  # Dibuja landmarks en el frame de video original
                mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        if len(sequence) == sequence_length:  # Cuando se completa una secuencia
            np.save(os.path.join(save_dir, f"secuencia_{counter}.npy"), sequence)  # Guarda la secuencia en un archivo .npy
            counter += 1  # Incrementa el contador de secuencias
            sequence = []  # Reinicia la lista para la próxima secuencia
            print(f"Secuencias capturadas: {counter}/{num_sequences}")  # Muestra progreso


        # Mostrar información en ambas ventanas
        info_text = f"Secuencias: {counter}/{num_sequences}"  # Texto con el progreso actual
        cv2.putText(frame, info_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)  # Superpone texto en el frame
        cv2.putText(landmark_canvas, info_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)  # Texto en el canvas

        cv2.imshow("Recolección de Datos", frame)  # Muestra el frame original con landmarks
        cv2.imshow(landmark_window_name, landmark_canvas)  # Muestra el canvas con solo los landmarks
        if cv2.waitKey(1) & 0xFF == 27 or counter >= num_sequences:  # Termina con ESC o al completar las secuencias
            break

    cap.release()  # Libera los recursos de la cámara
    cv2.destroyAllWindows()  # Cierra todas las ventanas de OpenCV
    gestures = get_existing_gestures()  # Actualiza la lista global de gestos
    print(f"\nSe recolectaron {counter} secuencias para el gesto '{gesture}'")  # Resumen final

## 6. CARGA DE DATOS

In [8]:
def tf_3d_rotation(angles):
    """Rotación 3D vectorizada para landmarks"""
    # Matriz de rotación alrededor del eje X (ángulo en radianes)
    rx = tf.convert_to_tensor([
        [1, 0, 0],  # Componente X sin rotar
        [0, tf.cos(angles[0]), -tf.sin(angles[0])],  # Componentes Y y Z afectadas por el ángulo
        [0, tf.sin(angles[0]), tf.cos(angles[0])]  # Componentes Y y Z rotadas
    ])
    
    # Matriz de rotación alrededor del eje Y
    ry = tf.convert_to_tensor([
        [tf.cos(angles[1]), 0, tf.sin(angles[1])],  # Componentes X y Z afectadas
        [0, 1, 0],  # Componente Y sin rotar
        [-tf.sin(angles[1]), 0, tf.cos(angles[1])]  # Componentes X y Z rotadas
    ])
    
    # Matriz de rotación alrededor del eje Z
    rz = tf.convert_to_tensor([
        [tf.cos(angles[2]), -tf.sin(angles[2]), 0],  # Componentes X y Y afectadas
        [tf.sin(angles[2]), tf.cos(angles[2]), 0],  # Componentes X y Y rotadas
        [0, 0, 1]  # Componente Z sin rotar
    ])
    
    # Combinación de rotaciones: Z -> Y -> X (multiplicación matricial en orden inverso)
    return tf.matmul(rz, tf.matmul(ry, rx))  # rz * ry * rx = rotación compuesta

def augment_landmarks(landmarks):
    """Aumentación 3D mejorada"""
    # Generar ángulos aleatorios en grados (-15° a 15°) y convertir a radianes
    angles = tf.random.uniform([3], -15.0, 15.0) * (np.pi / 180.0)
    
    # Obtener matriz de rotación compuesta
    rot_matrix = tf_3d_rotation(angles)
    
    # Aplicar rotación a los landmarks
    original_shape = tf.shape(landmarks)  # Guardar forma original (ej: [secuencia, landmarks, coordenadas])
    landmarks = tf.reshape(landmarks, [-1, 3])  # Aplanar a matriz 2D: (N*M, 3) donde N=secuencias, M=landmarks
    landmarks = tf.matmul(landmarks, rot_matrix)  # Rotar cada punto 3D
    landmarks = tf.reshape(landmarks, original_shape)  # Restaurar forma original
    
    # Añadir ruido gaussiano para mayor variabilidad
    noise = tf.random.normal(tf.shape(landmarks), mean=0.0, stddev=0.08)  # Ruido con desviación estándar 0.08
    return landmarks + noise  # Landmarks rotados + ruido (aumentación combinada)

**Explicación clave:**
1. `tf_3d_rotation`: Crea matrices de rotación 3D para cada eje y las combina para generar una transformación espacial compuesta.
2. `augment_landmarks`: Aplica transformaciones aleatorias a los landmarks para:
   - Mejorar la generalización del modelo
   - Simular variaciones en la captura de movimientos
   - Prevenir sobreajuste mediante aumento de datos
3. **Flujo completo:** Rotación 3D + ruido = transformaciones realistas que mantienen la estructura anatómica pero introducen variabilidad.

In [9]:
def custom_augmentation(sequence):
    """Aumentación 100% en TensorFlow con rotaciones 3D"""
    sequence = tf.cast(sequence, tf.float32)  # Conversión a float32 para operaciones de TensorFlow
    
    # 1. Aplicar rotación 3D y ruido base (definido en augment_landmarks)
    sequence = augment_landmarks(sequence)  # Aplica rotación 3D y ruido inicial
    
    # 2. Ruido Gaussiano adicional para mayor variabilidad
    noise = tf.random.normal(tf.shape(sequence), mean=0.0, stddev=0.05)  # Genera ruido con desviación estándar 5%
    sequence = tf.add(sequence, noise)  # Suma el ruido a la secuencia
    
    # 3. Escalado aleatorio para simular variaciones de tamaño
    scale_factor = tf.random.uniform([], 0.9, 1.1)  # Factor de escala entre 90% y 110%
    sequence = tf.multiply(sequence, scale_factor)  # Aplica escalado a todos los puntos
    
    return sequence  # Devuelve secuencia aumentada

def create_dataset(X_data, y_data, augment=False):
    """Crea pipeline de datos optimizado para entrenamiento"""
    dataset = tf.data.Dataset.from_tensor_slices((X_data, y_data))  # Crea dataset básico
    
    if augment:  # Solo aplica aumentación si está activado
        dataset = dataset.map(
            lambda x, y: (custom_augmentation(x), y),  # Aplica aumentación a features manteniendo labels
            num_parallel_calls=tf.data.AUTOTUNE  # Paralelismo automático
        )
        dataset = dataset.shuffle(1000)  # Mezcla datos con buffer de 1000 elementos
    
    # Configuración final del pipeline
    return dataset.batch(32).prefetch(tf.data.AUTOTUNE)    # Lotes de 32 muestras # Precarga datos para maximizar throughput

**Explicación detallada:**

1. **`custom_augmentation`** (Aumentación de datos):
   - *Propósito:* Mejorar la generalización del modelo mediante transformaciones espaciales y numéricas.
   - *Flujo:*
     1. **Rotación 3D + Ruido base:** Aplica transformaciones espaciales realistas usando la función `augment_landmarks`.
     2. **Ruido adicional:** Añade variabilidad numérica extra (5% de desviación estándar).
     3. **Escalado:** Simula cambios de tamaño (+/-10%) manteniendo proporciones anatómicas.

2. **`create_dataset`** (Pipeline de datos):
   - *Optimizaciones clave:*
     - **Paralelismo inteligente:** `num_parallel_calls=tf.data.AUTOTUNE` permite que TensorFlow optimice automáticamente el uso de CPU.
     - **Prefetching:** `prefetch` solapa la preparación de datos y el entrenamiento para evitar cuellos de botella.
     - **Mezcla estratégica:** `shuffle(1000)` mantiene diversidad en lotes sin consumo excesivo de memoria.

3. **Efecto combinado:**
   - Genera **variaciones sintéticas** de los gestos originales.
   - **Triplica la diversidad efectiva** del dataset sin recolectar nuevos datos.
   - **Mejora resistencia** a:
     - Cambios de perspectiva
     - Tamaños de mano variables
     - Ruido en detección de landmarks

**Ejemplo de transformación:**
Un gesto original de "A" se convierte en:
- Versión rotada 12° a la derecha
- Con landmarks ligeramente desplazados (ruido)
- Escalado al 95% de tamaño original
→ Todo manteniendo la etiqueta "A" para aprendizaje supervisado.

In [10]:
def load_data(augment=True):
    X = []  # Lista para almacenar secuencias de landmarks
    y = []  # Lista para almacenar etiquetas numéricas
    
    for label_idx, gesture in enumerate(gestures):  # Iterar sobre cada gesto con su índice
        gesture_dir = os.path.join(dataset_dir, gesture)  # Ruta completa al directorio del gesto
        sequences = [f for f in os.listdir(gesture_dir) if f.endswith('.npy')]  # Listar archivos .npy
        
        print(f"Gesto '{gesture}' - secuencias encontradas: {len(sequences)}")  # Debug: mostrar cantidad
        
        for seq_file in sequences:  # Procesar cada secuencia
            seq_path = os.path.join(gesture_dir, seq_file)  # Ruta completa al archivo
            sequence = np.load(seq_path)  # Cargar secuencia desde disco
            
            if sequence.shape == (sequence_length, total_landmarks):  # Verificar dimensiones correctas
                X.append(sequence)  # Agregar secuencia válida
                y.append(label_idx)  # Agregar etiqueta correspondiente
            else:
                print(f"Secuencia {seq_file} con forma {sequence.shape} ignorada.")  # Advertencia formato incorrecto
    
    return np.array(X, dtype=np.float32), np.array(y), gestures  # Devolver datos como arrays numpy


**Explicación línea por línea:**

1. **`def load_data(augment=True):`**  
   - *Función principal* para cargar datos desde disco  
   - `augment`: Parámetro no utilizado (posible herencia de versión anterior)

2. **`X = []`, `y = []`**  
   - Inicializa listas vacías:  
     - `X`: Almacenará secuencias de landmarks (features)  
     - `y`: Almacenará índices numéricos de los gestos (labels)

3. **`for label_idx, gesture in enumerate(gestures):`**  
   - Itera sobre cada gesto disponible  
   - `label_idx`: Índice numérico del gesto (0 para "A", 1 para "B", etc.)  
   - `gesture`: Nombre del gesto (ej: "A")

4. **`gesture_dir = os.path.join(...)`**  
   - Construye la ruta al directorio del gesto (ej: "dataset/A")

5. **`sequences = [f for f ...]`**  
   - Lista todos los archivos .npy en el directorio  
   - Cada archivo representa una secuencia guardada

6. **`print(f"Gesto '{gesture}'...`**  
   - Muestra diagnóstico de cuántas secuencias se encontraron  
   - Útil para detectar gestos con pocos datos

7. **`for seq_file in sequences:`**  
   - Procesa cada archivo de secuencia individualmente

8. **`seq_path = os.path.join(...)`**  
   - Obtiene la ruta completa al archivo .npy

9. **`sequence = np.load(seq_path)`**  
   - Carga la secuencia desde el archivo numpy  
   - Formato esperado: (sequence_length, total_landmarks)

10. **`if sequence.shape == (...)`**  
    - Verificación crítica de dimensiones:  
      - `sequence_length`: Número de frames por secuencia (ej: 90)  
      - `total_landmarks`: Landmarks por frame (ej: 126 = 2 manos × 21 landmarks × 3 coordenadas)

11. **`X.append(sequence)`**  
    - Agrega secuencia válida al conjunto de features

12. **`y.append(label_idx)`**  
    - Agrega el índice numérico correspondiente al gesto

13. **`else: print(...)`**  
    - Manejo de errores: secuencias con formato incorrecto se ignoran  
    - Previene problemas de dimensiones en el modelo

14. **`return np.array(...), ...`**  
    - Retorna:  
      - `X`: Array de secuencias (float32 para eficiencia en GPUs)  
      - `y`: Array de etiquetas  
      - `gestures`: Lista original de nombres de gestos

**Flujo de trabajo:**  
1. Recorre estructura de directorios del dataset  
2. Carga y valida cada secuencia guardada  
3. Construye arrays estructurados para entrenamiento  
4. Asegura consistencia en tipos de datos y dimensiones

**Ejemplo de salida:**  
Para un dataset con 3 gestos ("A", "B", "C") y 50 secuencias cada uno:  
```python
X.shape → (150, 90, 126)  # 150 muestras, 90 frames, 126 landmarks
y.shape → (150,)           # 150 etiquetas (0,1,2)
gestures → ["A", "B", "C"]
```

## 7. ENTRENAMIENTO DEL MODELO

In [11]:
def representative_dataset_gen():  
    # Generador de datos de ejemplo para calibración de cuantización
    for _ in range(100):  # Generar 100 muestras de calibración
        yield [  # Devolver lote de entrada en formato esperado por el modelo
            np.random.rand(  # Datos sintéticos con distribución uniforme [0, 1)
                1,  # Tamaño de batch (1 muestra por lote)
                sequence_length,  # Número de frames por secuencia (ej: 90)
                total_landmarks  # Total de landmarks 3D (ej: 126 = 21 landmarks/mano * 2 manos * 3 coordenadas)
            ).astype(np.float32)  # Tipo requerido para la conversión TFLite
        ]


def train_model():
    global X_mean, X_std, gestures  # Accede a variables globales para normalización y gestos
    global MODEL_PATH, NORMALIZATION_PARAMS_PATH  

    # 1. Verificación inicial de datos
    gestures = get_existing_gestures()  # Obtiene lista de gestos desde el directorio
    if not gestures:  # Si no hay gestos detectados
        print("\nNo hay datos recolectados. Primero recolecte datos de gestos.")
        return  # Detiene la ejecución

    # 2. Carga y preparación de datos
    print("\nCargando datos y preparando el entrenamiento...")
    X, y, gestures = load_data(augment=False)  # Carga datos sin aumentación inicial
    y = tf.keras.utils.to_categorical(y)  # Convierte etiquetas a one-hot encoding
    
    # 3. División de datos en entrenamiento/validación
    X_train, X_val, y_train, y_val = train_test_split(X, y, 
        test_size=0.2,  # 20% para validación
        stratify=y,  # Mantiene distribución de clases
        random_state=42  # Semilla para reproducibilidad
    )

    # 4. Normalización de datos
    X_mean = np.mean(X_train, axis=(0, 1)).astype(np.float32)  # Media de entrenamiento
    X_std = np.std(X_train, axis=(0, 1)).astype(np.float32)  # Desviación estándar
    X_train = (X_train - X_mean) / X_std  # Normaliza datos de entrenamiento
    X_val = (X_val - X_mean) / X_std  # Aplica misma normalización a validación

    # Creación de datasets con pipeline optimizado
    train_dataset = create_dataset(X_train, y_train, augment=True)  # Dataset con aumentación
    val_dataset = create_dataset(X_val, y_val, augment=False)  # Validación sin aumentación
    

    # 5. Guardado de parámetros de normalización
    np.savez(NORMALIZATION_PARAMS_PATH, mean=X_mean, std=X_std)  # Guarda en archivo .npz
    
    # 6. Arquitectura del modelo híbrido (Conv + Attention)
    inputs = tf.keras.Input(shape=(sequence_length, total_landmarks))  # Capa de entrada
    
    # Mecanismo de atención espacial
    attention = tf.keras.layers.MultiHeadAttention(
        num_heads=4,  # 4 cabezas de atención
        key_dim=64  # Dimensión de claves/valores
    )(inputs, inputs)  # Auto-atención
    
    x = tf.keras.layers.Concatenate()([inputs, attention])  # Conexión residual
    
    # Bloque convolucional 1
    x = tf.keras.layers.Conv1D(
        128, 5,  # 128 filtros, kernel de tamaño 5
        activation='relu', #funcion de activacion, recta lineal
        padding='same'  # Mantiene dimensiones
    )(x)
    x = tf.keras.layers.MaxPooling1D(2)(x)  # Reducción temporal

    # Bloque convolucional 2
    x = tf.keras.layers.Conv1D(
        64, 3,  # 64 filtros, kernel de tamaño 3
        activation='relu',
        padding='same'
    )(x)

    # Atención temporal
    x = tf.keras.layers.MultiHeadAttention(
        num_heads=2,  # 2 cabezas para temporal
        key_dim=32  # Dimensión reducida
    )(x, x)  # Atención sobre secuencia procesada

    # Reducción dimensional final
    x = tf.keras.layers.GlobalAveragePooling1D()(x)  # Pooling temporal

    # Capas densas para clasificación
    x = tf.keras.layers.Dense(64, activation='relu')(x)  # Capa oculta
    outputs = tf.keras.layers.Dense(
        len(gestures),  # Neuronas de salida = número de gestos
        activation='softmax'  # Clasificación multiclase
    )(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)  # Construcción del modelo completo

   # 7. Compilación del modelo
    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=0.0001,  # Tasa de aprendizaje baja
            global_clipnorm=1.0  # Evita gradientes explosivos
        ),
        loss='categorical_crossentropy',  # Función de pérdida para clasificación
        metrics=['accuracy'],  # Métrica principal
        weighted_metrics=['accuracy']  # Métricas adicionales
    )
    
    model.summary()  # Muestra resumen de arquitectura

    # 8. Entrenamiento del modelo
    print("\nIniciando entrenamiento...")
    history = model.fit(
        train_dataset,
        validation_data=val_dataset,  # Datos de validación
        epochs=50,  # 50 pasadas completas
        verbose=1  # Muestra progreso
    )

    # 9. Guardado del modelo entrenado
    model.save(model_path)  # Guarda en formato Keras
    print(f"\nModelo guardado en {model_path}")

    # 10. Conversión a TFLite para despliegue
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]  # Cuantización básica
    converter.representative_dataset = representative_dataset_gen  # Calibración
    converter.target_spec.supported_ops = [  # Operaciones soportadas
        tf.lite.OpsSet.TFLITE_BUILTINS,  # Ops nativas
        tf.lite.OpsSet.SELECT_TF_OPS  # Ops especiales de TF
    ]
    converter.target_spec.supported_types = [tf.int8]  # Cuantización a 8 bits
    converter._experimental_default_to_single_batch_in_tensor_list_ops = True  # Optimización

    try:  # Manejo de errores en conversión
        tflite_model = converter.convert()
        with open(MODEL_PATH, 'wb') as f:
            f.write(tflite_model)  # Escribe modelo cuantizado
        print("\nModelo TFLite exportado exitosamente")
    except Exception as e:
        print(f"\nError en conversión TFLite: {str(e)}")

    # 11. Métricas finales
    val_accuracy = history.history['val_accuracy'][-1]  # Última precisión de validación
    print(f"Precisión de validación final: {val_accuracy:.2%}")

## TF LITE

In [12]:
def convert_to_tflite():  # Define la función para convertir el modelo entrenado a formato TFLite
    global MODEL_PATH, NORMALIZATION_PARAMS_PATH
    try:  # Inicia un bloque try para capturar errores en la conversión
        # Cargar el modelo entrenado
        model = tf.keras.models.load_model(model_path)  # Carga el modelo entrenado desde la ruta especificada en 'model_path'
        
        # Configurar el conversor con parámetros especiales
        converter = tf.lite.TFLiteConverter.from_keras_model(model)  # Crea un conversor TFLite a partir del modelo cargado
        
        # Añadir estas 3 líneas clave para compatibilidad con LSTM
        converter.target_spec.supported_ops = [  # Especifica las operaciones que deben ser compatibles en el modelo convertido
            tf.lite.OpsSet.TFLITE_BUILTINS,  # Usa las operaciones estándar de TensorFlow Lite
            tf.lite.OpsSet.SELECT_TF_OPS  # Permite operaciones adicionales de TensorFlow que no están incluidas por defecto en TFLite
        ]
        converter._experimental_lower_tensor_list_ops = False  # Desactiva la optimización de listas de tensores, útil para modelos LSTM
        converter.allow_custom_ops = True  # Habilita operaciones personalizadas que puedan estar presentes en el modelo
        
        # Realizar la conversión
        tflite_model = converter.convert()  # Convierte el modelo de Keras a formato TFLite
        
        # Guardar el modelo cuantizado
        with open(MODEL_PATH, 'wb') as f:  # Abre un archivo en modo escritura binaria
            f.write(tflite_model)  # Guarda el modelo convertido en el archivo
        
        print("\n✅ Conversión a TFLite exitosa!")  # Mensaje de éxito en la conversión
        
    except Exception as e:  # Captura cualquier error que ocurra durante la conversión
        print(f"\n❌ Error en conversión: {str(e)}")  # Imprime el mensaje de error
        print("Posibles soluciones:")  # Lista de sugerencias para solucionar errores comunes
        print("1. Verifique que el modelo .h5 existe")  # Sugerencia 1: Comprobar que el archivo del modelo existe
        print("2. Actualice TensorFlow: pip install --upgrade tensorflow")  # Sugerencia 2: Actualizar TensorFlow
        print("3. Reinicie el runtime/kernel")  # Sugerencia 3: Reiniciar el entorno de ejecución (útil en Google Colab o Jupyter Notebook)

    global gestures  # Se usa la variable global 'gestures'
    gestures = get_existing_gestures()  # Se actualiza la lista de gestos disponibles después de la conversión
    print("Gestos cargados para evaluación:", gestures)  # Muestra los gestos detectados en el dataset

    print("Salida del modelo:", model.output_shape)  # Imprime la forma de salida del modelo, útil para depuración

"""
def representative_dataset_gen():  # Define una función generadora para proporcionar ejemplos al convertir el modelo
    # Generador de datos de ejemplo para calibración
    for _ in range(100):  # Itera 100 veces para generar ejemplos de entrada
        yield [np.random.randn(1, sequence_length, total_landmarks).astype(np.float32)]  
        # Genera un array de números aleatorios con forma (1, sequence_length, total_landmarks)
        # Lo convierte a tipo de datos float32, requerido por TensorFlow Lite"""


'\ndef representative_dataset_gen():  # Define una función generadora para proporcionar ejemplos al convertir el modelo\n    # Generador de datos de ejemplo para calibración\n    for _ in range(100):  # Itera 100 veces para generar ejemplos de entrada\n        yield [np.random.randn(1, sequence_length, total_landmarks).astype(np.float32)]  \n        # Genera un array de números aleatorios con forma (1, sequence_length, total_landmarks)\n        # Lo convierte a tipo de datos float32, requerido por TensorFlow Lite'

In [13]:
def extrapolate_sequence(seq_array):  # Define la función para extrapolar secuencias incompletas
    """Extrapolación lineal para secuencias incompletas"""  # Docstring que indica que esta función rellena secuencias incompletas con valores extrapolados
    
    last_valid = np.where(seq_array.sum(axis=1) != 0)[0]  # Encuentra los índices de las filas que contienen datos (es decir, que no son completamente ceros)
    if len(last_valid) < 2:  # Si hay menos de 2 filas con datos, no se puede extrapolar una tendencia
        return seq_array  # Retorna la secuencia original sin modificaciones
    
    x = np.array([0, 1])  # Define los valores de x para la extrapolación (dos puntos de referencia)
    
    for col in range(seq_array.shape[1]):  # Itera sobre cada columna (cada coordenada de los landmarks)
        y = seq_array[last_valid[-2:], col]  # Toma los dos últimos valores válidos en la columna actual
        coeffs = np.polyfit(x, y, 1)  # Calcula los coeficientes de una regresión lineal de primer grado (pendiente e intercepto)
        seq_array[last_valid[-1]+1:, col] = np.polyval(coeffs, np.arange(2, 2+len(seq_array)-last_valid[-1]-1))  
        # Aplica la ecuación de la recta para extrapolar valores desde el último punto válido en adelante
    
    return seq_array  # Retorna la secuencia con los valores extrapolados


### **Explicación detallada:**
1. **Identificación de datos válidos:**  
   - Se usa `np.where(seq_array.sum(axis=1) != 0)[0]` para encontrar las filas que contienen valores distintos de cero.
   - Si hay menos de 2 valores no nulos, se devuelve la secuencia sin cambios, ya que no es posible hacer una extrapolación.

2. **Definición de puntos para regresión lineal:**  
   - `x = np.array([0, 1])` define dos posiciones de referencia para la extrapolación.

3. **Aplicación de regresión lineal por columna:**  
   - Para cada columna (correspondiente a una coordenada de los landmarks), se toman los dos últimos valores válidos.
   - `np.polyfit(x, y, 1)` ajusta una línea recta a estos dos puntos, generando coeficientes para la ecuación de la recta `y = mx + b`.

4. **Extrapolación de valores faltantes:**  
   - Se usa `np.polyval(coeffs, np.arange(2, 2+len(seq_array)-last_valid[-1]-1))` para calcular valores predichos basados en la ecuación de la recta y rellenar los datos faltantes.

### **Ejemplo práctico:**
Supongamos que tenemos una secuencia donde los últimos dos valores son conocidos:

```python
seq_array = np.array([
    [1, 2],
    [3, 4],
    [0, 0],  # Falta un valor aquí
    [0, 0]   # Falta otro valor aquí
])
```

Después de ejecutar `extrapolate_sequence(seq_array)`, los valores extrapolados en la tercera y cuarta fila serán generados usando la tendencia de las dos primeras filas.

Esta técnica es útil en procesamiento de señales cuando faltan datos en una secuencia y se quiere mantener una continuidad basada en los valores previos.

## 8. EVALUACION DEL MODELO

In [14]:
def evaluate(): #encargada de realizar inferencias con el modelo en tiempo real.
    """ 
    Función para evaluar el modelo de reconocimiento de gestos en tiempo real.
    Procesa video de la cámara, detecta manos, y predice gestos basados en secuencias
    de landmarks de MediaPipe.
    """
    global gestures #variable global para actualizar la lista de gestos detectados en el dataset.
    gestures = get_existing_gestures() #se llama a la funcion, carga la lista de gestos disponibles en el dataset.

    # Constantes para mantener consistencia
    # Se definen constantes con las rutas del modelo TFLite y los parámetros de normalización.
    global MODEL_PATH 
    global NORMALIZATION_PARAMS_PATH  
    
    # 1. Verificar si existe el modelo
    #Se verifica si el modelo TFLite existe; si no, muestra un mensaje y finaliza la función
    if not os.path.exists(MODEL_PATH):
        print("\n¡Primero debe entrenar y convertir el modelo!")
        return
    
    # 2. Cargar parámetros y modelo
    # Se intenta cargar los parámetros de normalización desde un archivo .npz.
    try:
        with np.load(NORMALIZATION_PARAMS_PATH) as data:
            X_mean = data['mean']
            X_std = data['std']
            
        # Cargar el modelo TFLite
        interpreter = tf.lite.Interpreter(model_path=MODEL_PATH) #Se carga el modelo TFLite en un tf.lite.Interpreter
        interpreter.allocate_tensors() #Se asigna memoria para los tensores del modelo.

        #Se obtienen detalles de entrada y salida del modelo.
        input_details = interpreter.get_input_details()[0]
        output_details = interpreter.get_output_details()[0]

        # Se imprime la forma del tensor de salida para verificar compatibilidad.
        print("Output details shape:", output_details['shape'])
    # Si hay un error al cargar el modelo o los parámetros, se muestra un mensaje y se finaliza la función.
    except Exception as e:
        print(f"\nError crítico al cargar modelo: {str(e)}")
        return

    # 3. Configuración de cámara
    cap = UDPCamera()#Se inicializa una instancia de UDPCamera para recibir video.
    
    # 4. Variables de estado
    # Se configuran variables para manejar el buffer de datos (CircularBuffer) 
    # y almacenar información de gestos, confianza y latencia.
    buffer = CircularBuffer(sequence_length)  # Usar solo buffer para secuencias
    #buffer = CircularBuffer(sequence_length,total_landmarks)  # la continuacion de la expliacion de class circulabuffer (probar)
    prediction_history = deque(maxlen=15)     #el suavizado de predicciones (deque), Para suavizado de predicciones
    current_gesture = "Esperando..."
    current_confidence = 0.0
    latency = 0.0
    fps_counter = deque(maxlen=30)
    last_time = time.time()
    high_sensitivity_mode = False
    unknown_counter = 0

    # 5. Configurar afinidad de CPU para mejor rendimiento
    # Se intenta fijar la afinidad del proceso a los primeros dos núcleos de la CPU para mejorar el rendimiento.
    try:
        p = psutil.Process()
        p.cpu_affinity([0, 1])  # Hilo principal en núcleos 0-1
    except Exception as e:
        print(f"Advertencia: No se pudo configurar afinidad de CPU: {str(e)}")

    # 6. Bucle principal optimizado
    while True:
        # Medición de FPS/Se mide el FPS usando la diferencia de tiempo entre frames.
        current_time = time.time()
        fps_counter.append(1/(current_time - last_time + 1e-7))
        last_time = current_time

        # Se captura un frame de la cámara; si falla, se omite la iteración.
        ret, frame = cap.read()
        if not ret:
            continue

        # Procesar landmarks de manos
        # Se convierte el frame de BGR a RGB y se procesan landmarks con MediaPipe.
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb_frame)
        
        # Inicializar landmarks con ceros (para casos sin detección)
        # Se extraen landmarks de la detección y se rellenan con ceros si hay menos de los esperados.
        landmarks = [0.0] * total_landmarks
        
        # Extraer landmarks si hay manos detectadas
        if results.multi_hand_landmarks:
            hand_landmarks = []
            for hand in results.multi_hand_landmarks[:2]:  # Máximo 2 manos
                for lm in hand.landmark:
                    hand_landmarks.extend([lm.x, lm.y, lm.z])
            
            # Rellenar con ceros si es necesario
            if len(hand_landmarks) < total_landmarks:
                hand_landmarks += [0.0] * (total_landmarks - len(hand_landmarks))
            else:
                hand_landmarks = hand_landmarks[:total_landmarks]  # Truncar si excede
                
            landmarks = hand_landmarks

        # Añadir al buffer circular /Se añade la secuencia al buffer circular.
        buffer.add(landmarks)

        #NUEVO EXPIREMNTAL, SE PUEDE ELIMINAR SI INTERFIERE
        #aplico la extrapolacion, la funcion declarada anteriormente 
        if not buffer.full:
            current_sequence = buffer.get_sequence()
            extrapolated_sequence = extrapolate_sequence(current_sequence)
            # Opcional: reemplazar el buffer con la secuencia extrapolada
            buffer.buffer[:len(current_sequence)] = extrapolated_sequence
            
        # Ajustar modo alta sensibilidad si está activado
        if high_sensitivity_mode:
            frame = cv2.resize(frame, (320, 240))  # Reducir resolución para aumentar velocidad
        
        #Inferencia del modelo
        # Realizar predicción cuando el buffer esté completo
        # Se obtiene una secuencia completa del buffer, se normaliza y se pasa al modelo TFLite.
        if buffer.full:
            try:
                # Obtener secuencia y preprocesar
                seq_array = buffer.get_sequence()
                seq_array = (seq_array - X_mean) / (X_std + 1e-7)
                input_data = seq_array.reshape(1, sequence_length, total_landmarks).astype(np.float32)
                
                # Verificar forma de los datos de entrada
                if input_data.shape != tuple(input_details['shape']):
                    print(f"Error: Forma esperada {input_details['shape']}, obtenida {input_data.shape}")
                    continue
                
                # Inferencia con medición de latencia
                start_time = time.perf_counter()
                interpreter.set_tensor(input_details['index'], input_data)
                interpreter.invoke()
                prediction = interpreter.get_tensor(output_details['index'])[0]

                latency = (time.perf_counter() - start_time) * 1000  # Se mide la latencia de la inferencia en ms
                """
                # Sistema de fallback para gestos desconocidos
                # Si la confianza es baja en varias iteraciones, cambia entre modos normal y de alta sensibilidad.
                if np.max(prediction) < 0.5:
                    unknown_counter += 1
                    if unknown_counter >= 3:
                        high_sensitivity_mode = not high_sensitivity_mode
                        print(f"Cambiando a modo: {'Alta Sensibilidad' if high_sensitivity_mode else 'Normal'}")
                        unknown_counter = 0
                else:
                    unknown_counter = 0
                """
                # Suavizado temporal con promedio de predicciones recientes
                # Se suaviza la predicción y se decide si la confianza es suficiente para mostrar el gesto detectado.
                prediction_history.append(prediction)
                smoothed_pred = np.mean(prediction_history, axis=0)
                predicted_idx = np.argmax(smoothed_pred)
                confidence = smoothed_pred[predicted_idx]
                
                # Umbral dinámico según modo de sensibilidad
                confidence_threshold = 0.7 if not high_sensitivity_mode else 0.5
                if confidence > confidence_threshold:
                    current_gesture = gestures[predicted_idx]
                    current_confidence = confidence
                else:
                    #current_gesture = "Desconocido"
                    current_confidence = 0.0

            except Exception as e:
                print(f"Error en predicción: {str(e)}")
                continue

        # Visualización de métricas en pantalla
        fps = np.mean(fps_counter) if fps_counter else 0
        
        #Cv2 configuracion:
        """cv2.putText(
            frame,                             # 1. Imagen donde se dibujará (matriz NumPy)
            f"Prediccion: {current_gesture}",  # 2. Texto a mostrar (cadena formateada)
            (10, 30),                          # 3. Posición (x,y) desde esquina superior izquierda
            cv2.FONT_HERSHEY_SIMPLEX,          # 4. Tipo de fuente (estilo de letra)
            0.9,                               # 5. Escala de fuente (tamaño relativo)
            (0, 0, 255),                       # 6. Color en formato BGR (Azul, Verde, Rojo)
            2                                  # 7. Grosor del texto en píxeles
        )"""

        
        # Información de rendimiento
        cv2.putText(frame, f"FPS: {fps:.1f}", (10, 110), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
        cv2.putText(frame, f"Latencia: {latency:.1f}ms", (10, 140), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
        
        # Información de modo
        mode_color = (0, 255, 0) if not high_sensitivity_mode else (0, 0, 255)
        cv2.putText(frame, f"Modo: {'Normal' if not high_sensitivity_mode else 'Alta Sensibilidad'}", 
                   (10, 170), cv2.FONT_HERSHEY_SIMPLEX, 0.6, mode_color, 2)
        
        # Información de predicción
        cv2.putText(frame, f"Prediccion: {current_gesture}", (10, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2) #BGR
        cv2.putText(frame, f"Confianza: {current_confidence:.2%}", (10, 70),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        # Visualización y salida
        # Mostrar frame y detectar tecla ESC para salir
        cv2.imshow("Predicciones en Tiempo Real", frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break

    # Liberar recursos
    cap.release()
    cv2.destroyAllWindows()

La función `evaluate()` es responsable de ejecutar en tiempo real el reconocimiento de gestos a partir de secuencias de video recibidas por UDP. A lo largo de su ejecución, realiza múltiples tareas como cargar el modelo, procesar el video, extraer landmarks de las manos, realizar predicciones con el modelo de aprendizaje profundo y mostrar los resultados en pantalla.  

A continuación, se analiza detalladamente cada bloque de la función, explicando su propósito y su funcionamiento.  

---

## **1. Inicio de la función y carga del modelo**
El primer paso de `evaluate()` es cargar el modelo TFLite y los parámetros de normalización necesarios para preprocesar los datos antes de la inferencia.  

### **1.1 Carga de la lista de gestos**
```python
global gestures
gestures = get_existing_gestures()
```
- `gestures` es una variable global que almacena los nombres de los gestos registrados en el dataset.  
- `get_existing_gestures()` busca en el directorio de datos los gestos previamente recolectados y devuelve una lista ordenada con los nombres de las clases.  

### **1.2 Definición de rutas del modelo y verificación de su existencia**
```python
MODEL_PATH = "model_quantized_90_4.tflite"
NORMALIZATION_PARAMS_PATH = 'normalization_params_90_4.npz'

if not os.path.exists(MODEL_PATH):
    print("\n¡Primero debe entrenar y convertir el modelo!")
    return
```
- `MODEL_PATH` almacena la ubicación del modelo TFLite previamente entrenado.  
- `NORMALIZATION_PARAMS_PATH` almacena los parámetros de normalización del dataset.  
- `os.path.exists(MODEL_PATH)` verifica si el modelo existe; si no, se muestra un mensaje de error y se detiene la función con `return`.  

### **1.3 Carga de los parámetros de normalización y del modelo TFLite**
```python
try:
    with np.load(NORMALIZATION_PARAMS_PATH) as data:
        X_mean = data['mean']
        X_std = data['std']
```
- `np.load(NORMALIZATION_PARAMS_PATH)` abre el archivo que contiene los valores de la media (`X_mean`) y la desviación estándar (`X_std`) de los datos de entrenamiento.  
- Estos valores son necesarios para normalizar las nuevas entradas antes de pasarlas al modelo.  

```python
    interpreter = tf.lite.Interpreter(model_path=MODEL_PATH)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()[0]
    output_details = interpreter.get_output_details()[0]
    print("Output details shape:", output_details['shape'])
```
- `tf.lite.Interpreter(model_path=MODEL_PATH)` carga el modelo TFLite en un intérprete de TensorFlow Lite.  
- `allocate_tensors()` asigna memoria a los tensores del modelo.  
- `get_input_details()` y `get_output_details()` extraen información sobre la estructura de los tensores de entrada y salida.  

Si ocurre un error en la carga del modelo, se captura y se imprime el mensaje:  
```python
except Exception as e:
    print(f"\nError crítico al cargar modelo: {str(e)}")
    return
```
Esto evita que el programa continúe si el modelo no pudo cargarse correctamente.  

---

## **2. Configuración del sistema**
Antes de iniciar la evaluación, se configuran los componentes necesarios para capturar video y realizar la inferencia.  

### **2.1 Inicialización de la cámara UDP**
```python
cap = UDPCamera()
```
- Se crea una instancia de `UDPCamera()`, que es la clase encargada de recibir el video en tiempo real desde la red a través de paquetes UDP.  

### **2.2 Variables de estado**
```python
buffer = CircularBuffer(sequence_length)  # Usar solo buffer para secuencias
prediction_history = deque(maxlen=15)     # Para suavizado de predicciones
current_gesture = "Esperando..."
current_confidence = 0.0
latency = 0.0
fps_counter = deque(maxlen=30)
last_time = time.time()
high_sensitivity_mode = False
unknown_counter = 0
```
- **`buffer = CircularBuffer(sequence_length)`**: Se inicializa un buffer circular para almacenar secuencias de landmarks antes de enviarlas al modelo.  
- **`prediction_history = deque(maxlen=15)`**: Se utiliza una cola de tamaño 15 para almacenar las últimas predicciones y suavizar las salidas del modelo.  
- **`fps_counter = deque(maxlen=30)`**: Almacena las últimas 30 mediciones de FPS para calcular la tasa de actualización en tiempo real.  
- **`high_sensitivity_mode`**: Activa o desactiva un modo de alta sensibilidad cuando el modelo tiene dificultades para detectar gestos.  
- **`unknown_counter`**: Lleva el conteo de predicciones inciertas para alternar entre modos de sensibilidad.  

### **2.3 Optimización del rendimiento**
```python
try:
    p = psutil.Process()
    p.cpu_affinity([0, 1])  # Hilo principal en núcleos 0-1
except Exception as e:
    print(f"Advertencia: No se pudo configurar afinidad de CPU: {str(e)}")
```
- Se intenta fijar la ejecución del proceso en los núcleos 0 y 1 de la CPU para mejorar la eficiencia en sistemas multinúcleo.  
- Si el sistema no lo permite, se muestra una advertencia sin afectar la ejecución.  

---

## **3. Bucle principal de procesamiento**
La función entra en un bucle donde captura video, extrae landmarks, realiza predicciones y muestra los resultados.  

### **3.1 Cálculo de FPS**
```python
while True:
    current_time = time.time()
    fps_counter.append(1/(current_time - last_time + 1e-7))
    last_time = current_time
```
- Calcula la tasa de frames por segundo usando el tiempo transcurrido entre iteraciones.  

### **3.2 Captura de video**
```python
ret, frame = cap.read()
if not ret:
    continue
```
- Se obtiene un frame de la cámara UDP.  
- Si no se recibe un frame válido, se omite la iteración.  

### **3.3 Procesamiento de landmarks**
```python
rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
results = hands.process(rgb_frame)
```
- Convierte el frame a formato RGB y lo procesa con MediaPipe para detectar manos.  

Si se detectan manos, se extraen sus landmarks:  
```python
landmarks = [0.0] * total_landmarks

if results.multi_hand_landmarks:
    hand_landmarks = []
    for hand in results.multi_hand_landmarks[:2]:  
        for lm in hand.landmark:
            hand_landmarks.extend([lm.x, lm.y, lm.z])
            
    if len(hand_landmarks) < total_landmarks:
        hand_landmarks += [0.0] * (total_landmarks - len(hand_landmarks))
    else:
        hand_landmarks = hand_landmarks[:total_landmarks]  
        
    landmarks = hand_landmarks
```
- Si se detectan menos puntos de los esperados, se rellenan con ceros.  

Los landmarks se añaden al buffer circular:  
```python
buffer.add(landmarks)
```

### **3.4 Inferencia del modelo**
Cuando el buffer está lleno, se realiza la predicción:  
```python
if buffer.full:
    try:
        seq_array = buffer.get_sequence()
        seq_array = (seq_array - X_mean) / (X_std + 1e-7)
        input_data = seq_array.reshape(1, sequence_length, total_landmarks).astype(np.float32)
        
        interpreter.set_tensor(input_details['index'], input_data)
        interpreter.invoke()
        prediction = interpreter.get_tensor(output_details['index'])[0]
```
- La secuencia se normaliza y se pasa al modelo para obtener una predicción.  

### **3.5 Modo de alta sensibilidad**
Si la predicción es incierta en varias iteraciones, el sistema ajusta su modo de operación:  
```python
if np.max(prediction) < 0.5:
    unknown_counter += 1
    if unknown_counter >= 3:
        high_sensitivity_mode = not high_sensitivity_mode
        unknown_counter = 0
```

### **3.6 Suavizado de predicciones**
```python
prediction_history.append(prediction)
smoothed_pred = np.mean(prediction_history, axis=0)
predicted_idx = np.argmax(smoothed_pred)
confidence = smoothed_pred[predicted_idx]
```
- Se usa una media móvil para evitar fluctuaciones en las predicciones.  

---

## **4. Visualización y salida**
El resultado se muestra en pantalla con OpenCV, y el programa finaliza si se presiona ESC.  
```python
cv2.imshow("Predicciones en Tiempo Real", frame)
if cv2.waitKey(1) & 0xFF == 27:
    break

cap.release()
cv2.destroyAllWindows()
```

Esta implementación optimiza el rendimiento y la robustez del sistema para reconocimiento de gestos en tiempo real. 🚀

Las mejoras clave que he hecho a su evaluate() la función incluye:

* Manejo de ruta de modelo consistente: He definido constantes en la parte superior de la función tanto para el modelo como para los archivos de parámetros de normalización para garantizar la consistencia.

* Gestión de secuencias simplificada: He eliminado el redundante sequence deque y consistentemente utilizó el CircularBuffer para gestionar secuencias de puntos de referencia manuales.

* Procesamiento de punto de referencia más limpio: La extracción de puntos de referencia ahora es más sencilla, con un manejo adecuado de los casos en que hay demasiados o muy pocos puntos de referencia.

* Mejor manejo de errores: Se agregaron mensajes de error más específicos e implementaron un continue declaración para manejar errores durante la predicción para que el bucle no se bloquee.

* Código no utilizado eliminado: Se han eliminado el código comentado y las variables no utilizadas.

* Se agregaron comentarios más detallados: Cada sección de la función ahora tiene comentarios claros que explican lo que hace.

* Estructura mejorada: La función ahora sigue un flujo más lógico con secciones distintas para la inicialización, el procesamiento del bucle principal y la visualización.

* Prevención de errores: Se agregó un bloque de prueba excepto para la configuración de afinidad de la CPU, ya que esto podría no funcionar en todas las plataformas.

* Mejor gestión de marcos: Añadido a continue instrucción cuando falla la recuperación de fotogramas en lugar de procesar datos potencialmente no válidos.

* Retroalimentación visual para cambios de modo: Se agregó una declaración de impresión para informar cuando el sistema cambia entre los modos normal y de alta sensibilidad.

Esta función revisada debe ser más confiable, mantenible y consistente, al tiempo que conserva todas las características avanzadas de su implementación original.

## 10. MENU

In [15]:
# Menú principal
def main():
    init_system()
    
    while True:
        print("\n=== Sistema de Reconocimiento de Lenguaje de Señas ===")
        print("1. Detectar Manos")
        print("2. Recolectar Datos")
        print("3. Entrenar Modelo, y despues ir a convertir a TFlite")
        print("4. Evaluar")
        print("5. Convertir a TFLite")  # Nueva opción
        print("6. Salir")
        
        choice = input("\nSeleccione una opción: 1. Detectar Manos, 2. Recolectar Datos, 3. Entrenar Modelo, 4. Evaluar, 5. Convertir a TFLite, 6. Salir ")
        
        if choice == '1':
            detect_hands()
        elif choice == '2':
            collect_data()
        elif choice == '3':
            train_model()
        elif choice == '4':
            evaluate()
        elif choice == '5':  # Nueva opción de conversión
            convert_to_tflite()
        elif choice == '6':
            print("\n¡Hasta luego!")
            break
        else:
            print("\nOpción inválida. Por favor, intente de nuevo.")

# MENU

In [16]:
if __name__ == "__main__":
    main()


=== Sistema de Reconocimiento de Lenguaje de Señas ===
1. Detectar Manos
2. Recolectar Datos
3. Entrenar Modelo, y despues ir a convertir a TFlite
4. Evaluar
5. Convertir a TFLite
6. Salir
Output details shape: [1 5]
UDP Camera started on 0.0.0.0:5000
UDP Camera released
UDP Camera released

=== Sistema de Reconocimiento de Lenguaje de Señas ===
1. Detectar Manos
2. Recolectar Datos
3. Entrenar Modelo, y despues ir a convertir a TFlite
4. Evaluar
5. Convertir a TFLite
6. Salir

¡Hasta luego!
